In [76]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# simple van der Pol example
# assume that coeff are the estimated coefficients
coeff = np.zeros(19) # 1 noise, 
    # x, y, xx, xy, yy, xxx, xxy, xyy, yyy for dx/dt and dy/dt
coeff[0] = 0.05 # noise
coeff[1] = 0.11
coeff[2] = 1.1  #true
coeff[4] = -0.15 
coeff[10] = -0.95 #true
coeff[11] = 1.9 #true
coeff[14] = 0.12
coeff[16] = -2.1 #true

In [3]:
# approximate range of values
xmin = -2
xmax = 2
ymin = -4
ymax = 4

In [4]:
x, y = symbols("x y")

In [5]:
yy = y**2


In [6]:
coeff[1:].reshape(2,-1)[0]

array([ 0.11,  1.1 ,  0.  , -0.15,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ])

In [7]:
# compute the Drift of x
D1x = Integer(0)
sigma_x = (coeff[1:].reshape(2,-1))[0]
Poly = list([x, y])
Dim = 2
CoeffIndex = 0

for i in range(Dim): # linear
    print(Poly[i])
    D1x += sigma_x[CoeffIndex]*Poly[i]
    CoeffIndex +=1

for i in range(Dim): # quadratic
    for j in range(i,Dim):
        print(Poly[i]*Poly[j])
        D1x += sigma_x[CoeffIndex]*Poly[i]*Poly[j]
        CoeffIndex +=1
for i in range(Dim): # cubic
    for j in range(i,Dim):
        for k in range(j,Dim):
            print(Poly[i]*Poly[j]*Poly[k])
            D1x += sigma_x[CoeffIndex]*Poly[i]*Poly[j]*Poly[k]
            CoeffIndex +=1

x
y
x**2
x*y
y**2
x**3
x**2*y
x*y**2
y**3


In [8]:
D1x

-0.15*x*y + 0.11*x + 1.1*y

In [9]:
# compute the Drift of x
D1y = Integer(0)
sigma_y = (coeff[1:].reshape(2,-1))[1]
Poly = list([x, y])
Dim = 2
CoeffIndex = 0

for i in range(Dim): # linear
    print(Poly[i])
    D1y += sigma_y[CoeffIndex]*Poly[i]
    CoeffIndex +=1

for i in range(Dim): # quadratic
    for j in range(i,Dim):
        print(Poly[i]*Poly[j])
        D1y += sigma_y[CoeffIndex]*Poly[i]*Poly[j]
        CoeffIndex +=1
for i in range(Dim): # cubic
    for j in range(i,Dim):
        for k in range(j,Dim):
            print(Poly[i]*Poly[j]*Poly[k])
            D1y += sigma_y[CoeffIndex]*Poly[i]*Poly[j]*Poly[k]
            CoeffIndex +=1
D1y.simplify()

x
y
x**2
x*y
y**2
x**3
x**2*y
x*y**2
y**3


-2.1*x**2*y - 0.95*x + 0.12*y**2 + 1.9*y

In [10]:
len(Poly)

2

In [11]:
def GetPoly(List, Coeff, Constant = False):
    # takes List of (linear) base functions [x1, x2, x3,..] and 
    # their polynomial coefficients Coeff of third order
    # returns a sympy polynomial
    # if Constant = True: the first Coeff[0] is used as constant
    
    Out = Integer(0)
    Index = 0 # iterating over the Coefficients
    Dim = len(List)
    
    if Constant:
        Out += Coeff[Index]
        Index +=1
    
    for i in range(Dim): # linear
        Out += Coeff[Index] * List[i]
        Index +=1
    
    for i in range(Dim): # quadratic
        for j in range(i,Dim):
            Out +=Coeff[Index] * List[i] * List[j]
            Index +=1
            
    for i in range(Dim): # cubic
        for j in range(i,Dim):
            for k in range(j,Dim):
                Out += Coeff[Index] * List[i] * List[j] * List[k]
                Index +=1
    return(Out)

### For dx/dt

In [12]:
Polyx = GetPoly(Poly, sigma_x)
Polyx

-0.15*x*y + 0.11*x + 1.1*y

In [13]:
Polyx.subs({x:1, y:2})

2.01000000000000

In [14]:
integrate(Polyx,
          (x, xmin, xmax), 
          (y, ymin,ymax))

0

In [15]:
m_Polyx_x =  integrate(Polyx, # maringalised over x
                      (x, xmin, xmax)) / (xmax-xmin)
m_Polyx_y =  integrate(Polyx, # maringalised over y
                      (y, ymin, ymax)) / (ymax-ymin)

In [16]:
m_Polyx_x, m_Polyx_y

(1.1*y, 0.11*x)

In [17]:
(Polyx - m_Polyx_x)**2

0.0225*(-x*y + 0.733333333333333*x)**2

In [18]:
Causal_Polyx_x = integrate((Polyx - m_Polyx_x)**2, 
                          (x, xmin, xmax),
                           (y, ymin, ymax))
Causal_Polyx_y = integrate((Polyx - m_Polyx_y)**2, 
                          (x, xmin, xmax),
                           (y, ymin, ymax))

In [19]:
print(Causal_Polyx_x, Causal_Polyx_y)

5.63626666666667 211.626666666667


In [20]:
(xmax-xmin)*(ymax-ymin)

32

### And for dy/dt

In [21]:
Polyy = GetPoly(Poly, sigma_y)
pprint(Polyy)
m_Polyy_x =  integrate(Polyy, # maringalised over x
                      (x, xmin, xmax)) / (xmax-xmin)
m_Polyy_y =  integrate(Polyy, # maringalised over y
                      (y, ymin, ymax)) / (ymax-ymin)

       2                    2        
- 2.1⋅x ⋅y - 0.95⋅x + 0.12⋅y  + 1.9⋅y


In [22]:
Causal_Polyy_x = integrate((Polyy - m_Polyy_x)**2, 
                          (x, xmin, xmax),
                           (y, ymin, ymax))
Causal_Polyy_y = integrate((Polyy - m_Polyy_y)**2, 
                          (x, xmin, xmax),
                           (y, ymin, ymax))

In [24]:
print(Causal_Polyy_x, Causal_Polyy_y)

1108.92800000000 1219.14709333333


In [45]:
r = list([[-1,1], [-2,2], [0,1]])
print(type(r), len(r))
r[0]

<class 'list'> 3


[-1, 1]

## turn it into a function

In [54]:
GetPoly(symbols("x y"), sigma_x)

-0.15*x*y + 0.11*x + 1.1*y

In [124]:
def CheckInfluences(Function, ranges, names ):
    # Takes sympy Function with definition volume defined by ranges = list [[x0min, x0max],[x1min,x1max],...]
        # optional: names = list of sympy symbols of the variable names
    # returns the Causal Integral for marginalising over x0, for x1, for x2, ...
    Dimen = len(ranges)
    Output = pd.DataFrame({"Variable":(),
                           "Influence":()})
        
    for i in range(Dimen):
        
        Marginal = (ranges[i][1]-ranges[i][0])**(-1)* integrate(Function, 
                                                                (names[i], 
                                                                 ranges[i][0], 
                                                                 ranges[i][1]) )
        
        Integrand = (Marginal - Function)**2
        
        for j in range(Dimen):
            Integrand = integrate(Integrand, 
                                  (names[j],
                                   ranges[j][0], 
                                   ranges[j][1]))
        
        Name = names[i]
        
        Output = Output.append(pd.DataFrame({"Variable": [names[i]],
                                            "Influence": [Integrand]}),
                              ignore_index=True)
    return(Output)

In [125]:
str(symbols("x y")[1])

'y'

In [126]:
CheckInfluences(GetPoly(symbols("x y"), sigma_x), 
               [[xmin, xmax], [ymin, ymax]],
               symbols("x y"))

,Variable,Influence
0,x,5.63626666666667
1,y,211.626666666667


In [127]:
CheckInfluences(GetPoly(symbols("x y"), sigma_y), 
               [[xmin, xmax], [ymin, ymax]],
               symbols("x y"))

,Variable,Influence
0,x,1108.92800000000
1,y,1219.14709333333
